In [6]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [7]:
pc = Pinecone(api_key= os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension = 1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '3bffaea8518d78ae9968dc199f35aacf', 'Date': 'Sat, 24 Aug 2024 16:07:13 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Great teacher, explains concepts well but can be a bit strict on grading.'},
 {'professor': 'Dr. Jane Doe',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Amazing professor! Very approachable and makes complex topics easy to understand.'},
 {'professor': 'Prof. Albert Jones',
  'subject': 'Computer Science',
  'stars': 3,
  'review': 'Lectures are informative but tend to be dry. Assignments are challenging.'},
 {'professor': 'Dr. Emily Brown',
  'subject': 'History',
  'stars': 2,
  'review': 'Not very engaging. Lectures can be boring and exams are tough.'},
 {'professor': 'Prof. Michael Green',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Excellent teacher with a deep understanding of the subject. Highly recommended!'},
 {'professor': 'Dr. Sarah White',
  'subject': 'Chemistry',
  'stars': 3,
  'review': "The material is challenging, and the professor's explanations could be clearer."},
 

: 

In [9]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values" : embedding,
        "id" : review['professor'],
        "metadata": {
            "review" : review['review'],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })
    

In [11]:
processed_data[0]

{'values': [-0.016514499,
  0.008413282,
  -0.0039132996,
  -0.013181641,
  0.024740541,
  -9.904418e-06,
  -0.01045419,
  -0.009424374,
  -0.022855667,
  0.049680807,
  0.010572774,
  0.009049895,
  -0.018174687,
  -0.008825208,
  0.01899854,
  -0.027611542,
  -0.0073834667,
  -0.034052573,
  -0.0053113527,
  0.034726635,
  -0.003988195,
  -0.01756304,
  0.031381294,
  0.06316203,
  -0.014255146,
  -0.033028997,
  0.028635118,
  -0.021033205,
  0.054174546,
  0.019023504,
  0.06381112,
  -0.013830737,
  0.032155216,
  -0.048782058,
  -0.023791863,
  0.0318806,
  0.029259248,
  0.03614965,
  0.03872107,
  0.0011015907,
  -0.029259248,
  0.031506117,
  -0.024640681,
  -0.019497843,
  0.025327224,
  -0.013269019,
  -0.022955528,
  -0.029134423,
  -0.00276958,
  0.0193855,
  -0.03175577,
  0.05202754,
  0.050979,
  0.028560221,
  -0.012919506,
  0.012108136,
  -0.0036230788,
  0.0370484,
  0.04301509,
  0.0059791724,
  0.04541175,
  -0.034776565,
  -0.0013637256,
  0.00029821746,
  -0.010

In [14]:
index = pc.Index("rag")
index.upsert(
    vectors = processed_data,
    namespace = 'ns1'
)

{'upserted_count': 20}

In [15]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}